In [2]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore")

In [ ]:
semaphore = asyncio.Semaphore(20)
link_set = set()
cnt = 0
lock = asyncio.Lock()
async def get_link(session,i) :
    global cnt
    url =  f"https://oto.com.vn/mua-ban-xe/p{i}"
    async with semaphore:
        try :
            await asyncio.sleep(0.5)
            async with session.get(url,timeout = 15) as resp :
                html = await resp.text()
                soup = BeautifulSoup(html , "html.parser")
                links_tag = soup.find_all("h3", class_="title")
                for tag in links_tag:
                    a_tag = tag.find('a')
                    if a_tag and 'href' in a_tag.attrs:
                        link = "https://oto.com.vn" + a_tag['href']
                        link_set.add(link)
                async with lock:
                    cnt += 1
                    print(f"✅ Done: {i}")
        except Exception as e:
            print(f"⚠️ Error at page {i}: {e}")

async def main():
    async with aiohttp.ClientSession() as session :
        tasks = [get_link(session, i) for i in range(1, 100)]
        await asyncio.gather(*tasks)
await main()
print(f"\n📦 Total link: {len(link_set)}")

✅ Done: 50
✅ Done: 28
✅ Done: 18
✅ Done: 45
✅ Done: 32
✅ Done: 30
✅ Done: 12
✅ Done: 5
✅ Done: 8
✅ Done: 31
✅ Done: 43
✅ Done: 29
✅ Done: 44
✅ Done: 46
✅ Done: 33
✅ Done: 37
✅ Done: 48
✅ Done: 38
✅ Done: 40
✅ Done: 15
✅ Done: 7
✅ Done: 6
✅ Done: 9
✅ Done: 42
✅ Done: 13
✅ Done: 25
✅ Done: 26
✅ Done: 4
✅ Done: 17
✅ Done: 24
✅ Done: 23
✅ Done: 22
✅ Done: 47
✅ Done: 21
✅ Done: 16
✅ Done: 49
✅ Done: 39
✅ Done: 2
✅ Done: 20
✅ Done: 3
✅ Done: 41
✅ Done: 36
✅ Done: 35
✅ Done: 10
✅ Done: 27
✅ Done: 34
✅ Done: 14
✅ Done: 19
✅ Done: 11
✅ Done: 1
✅ Done: 70
✅ Done: 56
✅ Done: 96
✅ Done: 51
✅ Done: 98
✅ Done: 72
✅ Done: 77
✅ Done: 65
✅ Done: 68
✅ Done: 97
✅ Done: 95
✅ Done: 85
✅ Done: 80
✅ Done: 75
✅ Done: 78
✅ Done: 99
✅ Done: 83
✅ Done: 59
✅ Done: 57
✅ Done: 82
✅ Done: 94
✅ Done: 63
✅ Done: 79
✅ Done: 91
✅ Done: 84
✅ Done: 89
✅ Done: 67
✅ Done: 92
✅ Done: 53
✅ Done: 86
✅ Done: 66
✅ Done: 55
✅ Done: 81
✅ Done: 87
✅ Done: 73
✅ Done: 58
✅ Done: 93
✅ Done: 62
✅ Done: 52
✅ Done: 76
✅ Done: 69
✅ Done: 

In [ ]:
cnt = 0
lock = asyncio.Lock()
all_cars = []
semaphore = asyncio.Semaphore(20) 

async def detail(session, link):
    global cnt
    async with semaphore:
        try:
            async with session.get(link, timeout=10) as resp:
                if resp.status != 200:
                    print(f"❌ HTTP {resp.status} at {link}")
                    return

                html = await resp.text()
                soup = BeautifulSoup(html, "html.parser")
                car_info = {}

                name_tag = soup.find("div", class_="group-title-detail")
                name = name_tag.find("h1", class_="title-detail").get_text(strip=True) if name_tag else None
                car_info["Name"] = name

     
                price_tag = soup.find("span", class_="price")
                price = price_tag.get_text(strip=True) if price_tag else None
                car_info["Price"] = price


                labels = soup.find_all("label", class_="label")
                for label in labels:
                    key = label.get_text(strip=True)
                    value_node = label.find_next_sibling(text=True)

                    if not value_node or value_node.strip() == "":
                        div = label.find_next("div")
                        value = div.get_text(strip=True) if div else "N/A"
                    else:
                        value = value_node.strip()

                    car_info[key] = value

                all_cars.append(car_info)
                async with lock:
                    cnt += 1
                    print(f"✅ [{cnt}/{len(link_set)}] Done: {link}")

        except Exception as e:
            print(f"❌ ERROR at {link}: {e}")

async def main():
    async with aiohttp.ClientSession() as session:
        tasks = [detail(session, link) for link in link_set]
        await asyncio.gather(*tasks)

await main()

C:\Users\HP\AppData\Local\Temp\ipykernel_21012\4145975049.py:32: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  value_node = label.find_next_sibling(text=True)


✅ [1/1479] Done: https://oto.com.vn/mua-ban-xe-mercedes-benz-e300-ha-noi/e-class-2011-170000-km-aidxc23344784
✅ [2/1479] Done: https://oto.com.vn/mua-ban-xe-kia-k5-ha-noi/xe-chu-mua-moi-tu-dau-aidxc23338855
✅ [3/1479] Done: https://oto.com.vn/mua-ban-xe-mazda-cx-5-hai-phong/san-xuat-2022-aidxc23347014
✅ [4/1479] Done: https://oto.com.vn/mua-ban-xe-mercedes-benz-e250-ha-noi/nang-cap-tri-gia-gan-1-ty-them-nhieu-tinh-nang-cao-cap-chinh-hang-aidxc23340781
✅ [5/1479] Done: https://oto.com.vn/mua-ban-xe-mazda-2-ha-noi/hot-hot-hot-aidxc23342605
✅ [6/1479] Done: https://oto.com.vn/mua-ban-xe-hyundai-accent-hcm/xe-gia-dinh-chat-xe-dep-aidxc23344246
✅ [7/1479] Done: https://oto.com.vn/mua-ban-xe-ford-ecosport-binh-duong/xe-dep-nhu-moi-aidxc23027825
✅ [8/1479] Done: https://oto.com.vn/mua-ban-xe-mazda-3-ha-noi/mau-trang-bien-tu-dau-aidxc23335291
✅ [9/1479] Done: https://oto.com.vn/mua-ban-xe-honda-city-thanh-hoa/xe-dep-khong-loi-aidxc23336058
✅ [10/1479] Done: https://oto.com.vn/mua-ban-xe-toyota

CancelledError: 

In [5]:
semaphore = asyncio.Semaphore(50)
link_set2 = set()

async def get_link2(session, i):
    url = f"https://bonbanh.com/oto/page,{i}"
    async with semaphore:
        try:
            await asyncio.sleep(0.5)
            async with session.get(url, timeout=15) as resp:
                html = await resp.text()
                soup = BeautifulSoup(html, "html.parser")
                links_tag = soup.find_all("li", class_=["car-item row1", "car-item row2"])

                for tag in links_tag:
                    try:
                        a_tag = tag.find("a", itemprop="url")
                        if a_tag and a_tag.get("href"):
                            link = "https://bonbanh.com/" + a_tag.get("href")
                            link_set2.add(link)
                    except Exception as e:
                        print(f"⚠️ Error inside loop at page {i}: {e}")

                print(f"✅ Page {i} done.")

        except Exception as e:
            print(f"❌ ERROR at page {i}: {e}")

async def main():
    async with aiohttp.ClientSession() as session:
        tasks = [get_link2(session, i) for i in range(1, 150)]
        await asyncio.gather(*tasks)

await main()
print(f"\n📦 Total link: {len(link_set2)}")


✅ Page 28 done.
✅ Page 30 done.
✅ Page 42 done.
✅ Page 24 done.
✅ Page 44 done.
✅ Page 34 done.
✅ Page 9 done.
✅ Page 4 done.
✅ Page 50 done.
✅ Page 10 done.
✅ Page 8 done.
✅ Page 27 done.
✅ Page 46 done.
✅ Page 23 done.
✅ Page 2 done.
✅ Page 1 done.
✅ Page 35 done.
✅ Page 3 done.
✅ Page 6 done.
✅ Page 43 done.
✅ Page 7 done.
✅ Page 5 done.
✅ Page 38 done.
✅ Page 25 done.
✅ Page 15 done.
✅ Page 45 done.
✅ Page 22 done.
✅ Page 33 done.
✅ Page 21 done.
✅ Page 37 done.
✅ Page 20 done.
✅ Page 32 done.
✅ Page 19 done.
✅ Page 18 done.
✅ Page 26 done.
✅ Page 17 done.
✅ Page 14 done.
✅ Page 48 done.
✅ Page 47 done.
✅ Page 16 done.
✅ Page 49 done.
✅ Page 31 done.
✅ Page 13 done.
✅ Page 12 done.
✅ Page 41 done.
✅ Page 11 done.
✅ Page 39 done.
✅ Page 40 done.
✅ Page 29 done.
✅ Page 36 done.
✅ Page 51 done.
✅ Page 53 done.
✅ Page 60 done.
✅ Page 58 done.
✅ Page 95 done.
✅ Page 100 done.
✅ Page 62 done.
✅ Page 61 done.
✅ Page 91 done.
✅ Page 85 done.
✅ Page 97 done.
✅ Page 71 done.
✅ Page 80 done.


In [10]:
all_cars2 = []
cnt = 0
lock = asyncio.Lock()
semaphore = asyncio.Semaphore(50)

def in4(soup):
    specs = {}
    name_tag = soup.find("div", class_="title")
    name = name_tag.find_next("h1").get_text(strip=True) if name_tag else None
    specs["Name"] = name

    spec_section = soup.find("div", class_="box_car_detail")
    if spec_section:
        rows = spec_section.find_all("div", class_="row")
        for row in rows:
            label_div = row.find("div", class_="label")
            value_div = row.find("div", class_="txt_input")
            if label_div and value_div:
                label = label_div.get_text(strip=True).replace(":", "")
                value = value_div.get_text(strip=True)
                specs[label] = value
    return specs

async def detail2(session, link):
    global cnt
    async with semaphore:
        try:
            await asyncio.sleep(0.5)
            async with session.get(link, timeout=15) as resp:
                html = await resp.text()
                soup = BeautifulSoup(html, "html.parser")

                specs = in4(soup)
                all_cars2.append(specs)

                async with lock:
                    cnt += 1
                    print(f"✅ [{cnt}/{len(link_set2)}] Done: {link}")

        except Exception as e:
            print(f"❌ ERROR at {link}: {e}")

async def main():
    async with aiohttp.ClientSession() as session:
        tasks = [detail2(session, link) for link in link_set2]
        await asyncio.gather(*tasks)

await main()

✅ [1/2469] Done: https://bonbanh.com/xe-mazda-3-1.5-at-2018-6333414
✅ [2/2469] Done: https://bonbanh.com/xe-vinfast-vf9-eco-2025-5978621
✅ [3/2469] Done: https://bonbanh.com/xe-mercedes_benz-glc-300-4matic-2016-6345316
✅ [4/2469] Done: https://bonbanh.com/xe-volvo-xc90-recharge-t8-awd-2021-6330215
✅ [5/2469] Done: https://bonbanh.com/xe-mercedes_benz-maybach-gls-600-4matic-2022-6141946
✅ [6/2469] Done: https://bonbanh.com/xe-lexus-gx-550-luxury-2025-5890945
✅ [7/2469] Done: https://bonbanh.com/xe-lexus-gx-460-2015-6080645
✅ [8/2469] Done: https://bonbanh.com/xe-mercedes_benz-v_class-v250-luxury-2019-6294222
✅ [9/2469] Done: https://bonbanh.com/xe-mercedes_benz-c_class-c180-2020-6304950
✅ [10/2469] Done: https://bonbanh.com/xe-ford-territory-trend-1.5-at-2025-4631679
✅ [11/2469] Done: https://bonbanh.com/xe-lexus-rx-350l-2021-6277631
✅ [12/2469] Done: https://bonbanh.com/xe-mercedes_benz-glc-300-4matic-2021-6290766
✅ [13/2469] Done: https://bonbanh.com/xe-mitsubishi-pajero-sport-3.0g-4x